# Discrete Logarithm Problem

In [48]:
import dotenv
import numpy as np
from typing import List, Tuple

In [77]:

def gcd(a: int, b: int) -> int:
    """ Find GCD by recursing on a mod b """
    if(b == 0):
        return abs(a)
    else:
        return gcd(b, a % b)

def primes_below(n: int):
    """ """
    is_prime = [True, ] * (n + 1)
    for p in range(2, n):
        if is_prime[p]:
            for i in range(p * p, n + 1, p):
                is_prime[i] = False
            yield p
    return

def next_prime(n: int):
    """ """
    for p in primes_below((int(np.ceil(np.sqrt(n))) + 1) ** 2):
        if p > n:
            break
    return p

def is_prime(n: int) -> bool:
    """ check if is a prime by """
    for p in primes_below((int(np.ceil(np.sqrt(n))) + 1) ** 2):
        if p == n:
            return True
    return False

def ifactors(n: int):
    """   """
    half_n = n // 2
    is_prime = True
    for p in primes_below(half_n + 1):
        if p > half_n:
            break
        if n % p == 0:
            is_prime = False
            k = 1
            m = n // p
            while m % p == 0:
                m = m // p
                k += 1
            yield (p, k)
            
    if is_prime:
        yield (n, 1)
                
    return

def divisors(n: int) -> Tuple[int, ...]:
    """ """
    factors = tuple(ifactors(n))
    factor_primes = np.array([f[0] for f in factors] + [0,])
    # add one to every power, as we generate powers as (i mod factor_powers[j])
    factor_powers = np.array([f[1] for f in factors] + [0,]) + 1
    
    factors_count = len(factors)
    divisors_count = np.prod(factor_powers)

    # calc product of array of each prime factor to some power, varying from 0 to the max given from ifactors fn
    ds = sorted(int(np.prod([factor_primes[j] ** (i // np.prod(factor_powers[j - factors_count:]) % factor_powers[j])
                             for j in range(factors_count)]))
                for i in range(divisors_count))
    return tuple(ds)

def co_primes(n: int):
    """ """
    return set([a for a in range(1, n) if gcd(a, n) == 1])

def totient(n: int) -> int:
    """ Euler's phi function is the number of values less than a that are co-prime with n """
    return len(co_primes(n))

def order_of_powers(g: int, n: int) -> List[int]:
    """ g ^ k % n for k being co-prime with phi(n) """

    # order_of_powers = sorted(set([g ** k % n for k in co_primes(totient(n))]))
    # keep all calcs mod n to remove overflow errors
    ks = co_primes(totient(n))
    order_of_powers = set()
    g_k = 1
    for k in range(1, n):
        g_k = g_k * g  % n
        if k in ks:
            order_of_powers.add(g_k)
    return sorted(order_of_powers)

def order(a: int, n: int):
    """ Multiplicative order of a mod n is the smallest k for which a^k mod n is 1 """
    if a > n or gcd(a, n) != 1:
        return np.NaN

    a_k = 1
    for k in range(1, n):
        a_k = a_k * a  % n
        if a_k == 1:
            return k
    
    return np.NaN

def is_order_n(a: int, n: int):
    """ Multiplicative order of a mod n is the smallest k for which a^k mod n is 1 """
    if a > n or gcd(a, n) != 1:
        return np.NaN
    
    ord_n = totient(n)
    # we can do better than all k < n by only looking at divisors of totient(n)
    phi_n_divisors = divisors(ord_n)
    for k in phi_n_divisors:
        if a ** int(k) % n == 1:
            return k == ord_n
    
    return np.NaN

def cyclic_group(a, n, op):
    group = set([(a ** k) % n for k in range(1, n)])
    return group

def primative_roots(n):
    # g is a primitive root modulo n if for every integer a coprime to n, there is some integer k for which gk ≡ a (mod n)
    
    # check n is form 2, 4, p^s, 2p^s, where s is any positive integer and p is an odd prime
    factors = [f for f in ifactors(n)]
    if any((len(factors) < 1 or 2 < len(factors),  
            (len(factors) == 2 and (factors[0][0] != 2 or factors[0][1] > 1)),
            (len(factors) == 1 and factors[0][0] == 2 and factors[0][1] > 2),
            (len(factors) == 1 and factors[0][0] < 2))):
        return [] # Exception("No primitive roots exist")
    
    # find smallest primative root
    ord_n = totient(n)
    g = None
    for a in co_primes(n):
        if order(a, n) == ord_n:
            g = a
            break
    
    # There are phi(phi(n)) roots: return all roots using factors co-prime with phi(n)
    prime_roots = order_of_powers(g, n)
     
    assert len(prime_roots) == totient(ord_n)
    assert all(g ** ord_n % n == 1 for g in prime_roots)
    return prime_roots

In [78]:
#;order(4, 7
n = 601 # 334 # 25 # 334 # 109 # 
ord_n = totient(n)
ds = divisors(ord_n)

print(ds)
#print([d - d_1 for d, d_1 in zip(ds, [0,] + ds[:-1])])
#print([(k, 7 ** k, (7 ** k) % n, 7 ** int(k), (7 ** int(k)) % n) for k in ds])
#[1, 2, 3, 4, 5, 6, 8, 10, 12, 15, 20, 24, 25, 30, 40, 50, 60, 75, 100, 120, 150, 200, 300, 600]
print(order(2, n), is_order_n(2, n))
print(order(7, n), is_order_n(7, n))
print(3, 3 ** 600 % n,  order(3, n))
print(is_order_n(3, n))
print(7,  7 ** 600 % n,  order(7, n))
print(is_order_n(7, n))
#for a in co_primes(n):
#    if order(a, n) == ord_n:
#        print(a)
# [7, 11, 14, 19, 22, 29, 33, 34, 35, 38, 41, 42, 43, 44, 53, 55, 56, 58, 66, 68, 70, 73, 82, 84, 86, 88, 91, 93, 95, 103, 107, 110, 112, 114, 127, 136, 137, 142, 143, 145, 146, 152, 153, 155, 159, 161, 164, 168, 172, 173, 174, 177, 182, 186, 189, 190, 206, 212, 220, 224, 228, 232, 247, 249, 253, 254, 255, 257, 264, 265, 272, 274, 280, 283, 284, 286, 290, 291, 292, 297, 304, 309, 310, 311, 315, 317, 318, 321, 327, 329, 336, 337, 344, 346, 347, 348, 352, 354, 369, 373, 377, 381, 389, 395, 411, 412, 415, 419, 424, 427, 428, 429, 433, 437, 440, 442, 446, 448, 449, 455, 456, 458, 459, 464, 465, 474, 487, 489, 491, 494, 498, 506, 508, 510, 513, 515, 517, 519, 528, 531, 533, 535, 543, 545, 546, 548, 557, 558, 559, 560, 563, 566, 567, 568, 572, 579, 582, 587, 590, 594]
#
print(primative_roots(n))
print(order_of_powers(7, n))

(1, 2, 3, 4, 5, 6, 8, 10, 12, 15, 20, 24, 25, 30, 40, 50, 60, 75, 100, 120, 150, 200, 300, 600)
25 False
600 True
3 1 75
False
7 1 600
True
[7, 11, 14, 19, 22, 29, 33, 34, 35, 38, 41, 42, 43, 44, 53, 55, 56, 58, 66, 68, 70, 73, 82, 84, 86, 88, 91, 93, 95, 103, 107, 110, 112, 114, 127, 136, 137, 142, 143, 145, 146, 152, 153, 155, 159, 161, 164, 168, 172, 173, 174, 177, 182, 186, 189, 190, 206, 212, 220, 224, 228, 232, 247, 249, 253, 254, 255, 257, 264, 265, 272, 274, 280, 283, 284, 286, 290, 291, 292, 297, 304, 309, 310, 311, 315, 317, 318, 321, 327, 329, 336, 337, 344, 346, 347, 348, 352, 354, 369, 373, 377, 381, 389, 395, 411, 412, 415, 419, 424, 427, 428, 429, 433, 437, 440, 442, 446, 448, 449, 455, 456, 458, 459, 464, 465, 474, 487, 489, 491, 494, 498, 506, 508, 510, 513, 515, 517, 519, 528, 531, 533, 535, 543, 545, 546, 548, 557, 558, 559, 560, 563, 566, 567, 568, 572, 579, 582, 587, 590, 594]
[7, 11, 14, 19, 22, 29, 33, 34, 35, 38, 41, 42, 43, 44, 53, 55, 56, 58, 66, 68, 70, 73, 8

In [79]:


factors = tuple(f for p, s in factors for f in (p, ) * s)
print(factors)
f_primes = np.array([f[0] for f in factors] + [0,])
f_powers = np.array([f[1] for f in factors] + [0,])
f_div = tuple((np.prod((f_powers + 1)[j-len(factors):]), f_powers[j] + 1, f_primes[j]) for j in range(len(factors)))
print( f_primes, f_powers, f_div)
print( tuple((i , np.prod((f_powers + 1)[j-len(factors):]), f_powers[j] + 1, f_primes[j], (i // np.prod((f_powers + 1)[j-len(factors):])) % (f_powers[j] + 1)) for i in range(np.prod(f_powers + 1)) for j in range(0, len(factors))))
print( tuple([i, j , f_primes[j] ** ((i // np.prod((f_powers + 1)[j-len(factors):])) % (f_powers[j] + 1)) ] for i in range(np.prod(f_powers + 1)) for j in range(0, len(factors)) ))
print( tuple([(i, j , f_primes[j] ** ((i // np.prod((f_powers + 1)[j-len(factors):])) % (f_powers[j] + 1)))  for j in range(0, len(factors)) ] for i in range(np.prod(f_powers + 1)) ))
print( sorted(np.prod([(f_primes[j] ** ((i // np.prod((f_powers + 1)[j-len(factors):])) % (f_powers[j] + 1)))  for j in range(0, len(factors)) ]) for i in range(np.prod(f_powers + 1)) ))

#print( tuple(np.prod([f_primes[j] ** ((i // np.prod((f_powers + 1)[j-len(factors):])) % (f_powers[j] + 1)) ]) for i in range(np.prod(f_powers + 1)) for j in range(0, len(factors))))
#print(tuple((tuple((i % (f_powers[i] + 1)) for j in range(len(factors)) for i in range(np.prod(np.array([f[1] + 1 for f in factors])))))
#tuple(combinations(factors, 3))
#tuple(i for i in range(product(f[1] + 1 for f in factors)))
#print(tuple((i,) + tuple((i % (f[1] + 1)) for f in factors) for i in range(np.prod(np.array([f[1] + 1 for f in factors])))))
#print(tuple(tuple(f[0] ** (i % (f[1] + 1)) for f in factors) for i in range(np.prod(np.array([f[1] + 1 for f in factors])))))
#tuple(np.prod(np.array(tuple(f[0] ** (i % (f[1] + 1)) for f in factors))) for i in range(np.prod(np.array([f[1] + 1 for f in factors]))))

NameError: name 'factors' is not defined

In [42]:
n = 601
ord_n = totient(n)
fs = [f[0] for f in ifactors(ord_n)] + [ord_n, ]

print(sorted(set(order(a, n) for a in range(1, n)))) #, 2 ** ord_n % n
print([f for f in ifactors(ord_n)])

a = 10
print([(a, order(2, n)) + tuple(a ** k % n for k in fs) ]) 
print((a, ) + tuple(k for k in range(1, ord_n + 1) if a ** k % n == 1)) 

print(sorted(co_primes(totient(n))))
print(primitive_roots(n))

[1, 2, 3, 4, 5, 6, 8, 10, 12, 15, 20, 24, 25, 30, 40, 50, 60, 75, 100, 120, 150, 200, 300, 600]
[(2, 3), (3, 1), (5, 2)]
[(10, 25, 100, 399, 234, 1)]
(10, 300, 600)
[1, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 49, 53, 59, 61, 67, 71, 73, 77, 79, 83, 89, 91, 97, 101, 103, 107, 109, 113, 119, 121, 127, 131, 133, 137, 139, 143, 149, 151, 157, 161, 163, 167, 169, 173, 179, 181, 187, 191, 193, 197, 199, 203, 209, 211, 217, 221, 223, 227, 229, 233, 239, 241, 247, 251, 253, 257, 259, 263, 269, 271, 277, 281, 283, 287, 289, 293, 299, 301, 307, 311, 313, 317, 319, 323, 329, 331, 337, 341, 343, 347, 349, 353, 359, 361, 367, 371, 373, 377, 379, 383, 389, 391, 397, 401, 403, 407, 409, 413, 419, 421, 427, 431, 433, 437, 439, 443, 449, 451, 457, 461, 463, 467, 469, 473, 479, 481, 487, 491, 493, 497, 499, 503, 509, 511, 517, 521, 523, 527, 529, 533, 539, 541, 547, 551, 553, 557, 559, 563, 569, 571, 577, 581, 583, 587, 589, 593, 599]


NameError: name 'primitive_roots' is not defined

In [ ]:
len(cyclic_group(6, 17, lambda a, b: a * b))
n = 17
print(co_primes(n))
a_s = co_primes(n)
g = 6
set([(k, a, g ** k % n) for a in a_s  for k in range(1, n) if g ** k % n == a]), len(set([k for k in range(1, n) for a in a_s  if g ** k % n == a ])), len(a_s)

In [71]:
p = 17
a = 6
print([(k, a ** k % p)
       for k in range(1, p)])

print([(a, order(a, p)) for a in range(1, p)])
print(order_of_powers(a, p))
print(primative_roots(p))
len(cyclic_group(a, p, lambda a, b: a * b)), cyclic_group(a, p, lambda a, b: a * b)
#set([(a ** i % p) for i in range(1, p) if gcd(i, p-1) == 1])

[(1, 6), (2, 2), (3, 12), (4, 4), (5, 7), (6, 8), (7, 14), (8, 16), (9, 11), (10, 15), (11, 5), (12, 13), (13, 10), (14, 9), (15, 3), (16, 1)]
[(1, 1), (2, 8), (3, 16), (4, 4), (5, 16), (6, 16), (7, 16), (8, 8), (9, 8), (10, 16), (11, 16), (12, 16), (13, 4), (14, 16), (15, 8), (16, 2)]
{3, 5, 6, 7, 10, 11, 12, 14}
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}


(16, {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16})

In [46]:
p = 18
a = 5
print([(k, a ** k % p)
       for k in range(1, p)])

print([(a, order(a, p)) for a in range(1, p)])
print(order_of_powers(a, p))
print(primative_roots(p))
len(cyclic_group(a, p, lambda a, b: a * b)), cyclic_group(a, p, lambda a, b: a * b)
#set([(a ** i % p) for i in range(1, p) if gcd(i, p-1) == 1])

[(1, 5), (2, 7), (3, 17), (4, 13), (5, 11), (6, 1), (7, 5), (8, 7), (9, 17), (10, 13), (11, 11), (12, 1), (13, 5), (14, 7), (15, 17), (16, 13), (17, 11)]
[(1, 1), (2, 6), (3, 2), (4, 3), (5, 6), (6, 2), (7, 3), (8, 2), (9, 1), (10, 1), (11, 6), (12, 2), (13, 3), (14, 6), (15, 2), (16, 3), (17, 2)]
set()


(6, {1, 5, 7, 11, 13, 17})

In [5]:
print(primative_roots(25))


set()


In [58]:
f_p = np.array([1, 2, 3, 0])
(f_p + 1)[0-3:], (f_p + 1)[1-3:], (f_p + 1)[2-3:]

(array([3, 4, 1]), array([4, 1]), array([1]))